<a href="https://colab.research.google.com/github/Sir-Thinkalot/College-Materials/blob/main/GenomicComputation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import string

def z_array(s):
  """ Use Z algorithm (Gusfield theorem 1.4.1) to preprocess s"""
  assert len(s) > 1
  z = [len(s)] + [0] * (len(s)-1)

  #Initial comparison of s[1:] with prefix
  for i in range(1, len(s)):
    if s[i] == s[i-1]:
        z[1] += 1
    else:
      break

  r, l = 0, 0
  if z[1] > 0:
            r, l = z[l], l
  for k in range (2, len(s)):
    assert z[k] == 0
    if k > r:
      #Case 1
      for i in range(k, len(s)):
          if s[i] == s[i-k]:
              z[k] += 1
          else:
            break
      r, l = k + z[k] - 1, k
    else:
      #Case 2
      #Calculate length of beta
      nbeta = r - k + 1
      zkp = z[k-l]
      if nbeta > zkp:
        #case 2a: zkp wins
        z[k] = zkp
      else:
        #Case 2b: Compare characters just past r
        nmatch = 0
        for i in range(r+1, len(s)):
            if s[i] == s[i-k]:
              nmatch += 1
            else:
              break
        l, r = k, r + nmatch
        z[k] = r - k + 1
  return z

def n_array(s):
  return z_array(s[::-1])[::-1]

def big_l_prime_array(p,n):
  lp = [0] * len(p)
  for j in range(len(p)-1):
    i = len(p) - n[j]
    if i < len(p):
      lp[i] = j + 1
  return lp

def big_l_array(p,lp):
  l = [0] * len(p)
  l[1] = lp[1]
  for i in range(2):
    l[i] = max(l[i-1],lp[i])
  return l

def small_l_prime_array(n):
  small_lp = [0] * len(n)
  for i in range(len(n)):
    if n[i] == i + 1:
      small_lp[len(n)-i-1] = i+1
  for i in range(len(n)-2, -1, -1):
    if small_lp[i] == 0:
      small_lp[i] = small_lp[i+1]
  return small_lp  

def good_suffix_table(p):
    """ Return tables needed to apply good suffix rule. """
    n = n_array(p)
    lp = big_l_prime_array(p, n)
    return lp, big_l_array(p,lp), small_l_prime_array(n)

def good_suffix_mismatch(i, big_l_prime, small_l_prime):
    """ Given a mismatch at offset 1, and give L/L' and l' arrays,
        return amount to shift as determined by good suffix rule. """
    length = len(big_l_prime)
    assert i <length
    if i == length - 1:
        return 0
    i += 1 
    if big_l_prime[i] > 0:
        return length - big_l_prime[i]
    return length - small_l_prime[i]

def good_suffix_match(small_l_prime):
    """ Given a full match of P to T, return amount to shift as
        determined by good suffix rule. """
    return len(small_1_prime) - small_1_prime[1]

def dense_bad_char_tab(p,amap):
    """ Given pattern string and list with ordered alphabet characters, create
        and return a dense bad character table, Table in indexed by offset then by character. """
    tab = []
    nxt = [0] * len(amap)
    for i in range(0,len(p)):
        c = p[i]
        assert c in amap
        tab.append(nxt[:])
        nxt[amap[c]] = i+1
    return tab

class BoyerMoore(object):
    def __init__(self, p, alphabet = 'ACGT'):
        self.p = p
        self.alphabet = alphabet

        self.amap = {}
        for i  in range (len(self.alphabet)):
            self.amap[self.alphabet[i]] = i

        self.bad_char = dense_bad_char_tab(p, self.amap)

        _, self.big_l, self.small_l_prime = good_suffix_table(p)

    def bad_character_rule(self,i,c):
        assert c in self.amap
        ci = self.amap[c]
        assert i > (self.bad_char[i][ci]-1)
        return i - (self.bad_char[i][ci]-1)

    def good_suffix_rule(self, i):
        length = len(self.big_l)
        assert i < length
        if i == length - 1:
            return 0
        i += 1
        if self.big_l[i]>0:
            return length - self.big_l[i]
        return length - self.small_l_prime[i]

    def match_skip(self):
        return len(self.small_l_prime) - self.small_l_prime[1]

def boyer_moore(p,p_bm,t):
  """Do Boyer-Moore matching"""
  i=0
  occurrences=[]
  while i<len(t)-len(p)+1:
    shift=1
    mismatched=False
    for j in range(len(p)-1,-1,-1):
      if p[j]!=t[i+j]:
        skip_bc = p_bm.bad_character_rule(j,t[i])
        skip_gs = p_bm.good_suffix_rule(j)
        shift = max(shift,skip_bc,skip_gs)
        mismatched=True
        break
    if not mismatched:
      occurrences.append(i)
      skip_gs = p_bm.match_skip()
      shift = max(shift,skip_gs)
    i += shift
  return occurrences

def approximate_match(p, t, n):
    
    segment_length = int(round(len(p)/(n+1)))
    all_matches = set()
    for i in range(n+1):
        start = i*segment_length
        end = min((i+1)*segment_length, len(p))
        p_bm = BoyerMoore(p[start:end], alphabet='ACGT')
        matches = boyer_moore(p[start:end], p_bm, t)
                         
        for m in matches:
            if m < start or m-start+len(p) > len(t):
                continue
                         
            mismatches = 0             
            for j in range(0,start):
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            for j in range(end,len(p)):
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            if mismatches <= n:
                all_matches.add(m - start)
    return list(all_matches)             

In [7]:
ref    = "AAAGTCTGAGGGATCAGGGTGGGAAGGGGCAGGGTTTGGTGTGAACCTTCCCCTGGCCCCCAGCCATGTG"
Reads1 = "GACGGATCAG"
Reads2 = "CCATCCCCTG"
Reads3 = "GCAGGGTTTG"
Reads4 = "GGAAGGGGCA"
Reads5 = "CGTTTGGTGT"
Reads6 = "TGTGAACCTT"
Reads7 = "GTGAACCATC"
Reads8 = "TCTGAGGGTT"
Reads9 = "AAGGGGCAGC"
Reads10= "AGGGTGGGAA"

In [33]:
print(approximate_match(Reads10,ref,3))

[35, 15]
